In [ ]:
#  Licensed to the Apache Software Foundation (ASF) under one
#  or more contributor license agreements.  See the NOTICE file
#  distributed with this work for additional information
#  regarding copyright ownership.  The ASF licenses this file
#  to you under the Apache License, Version 2.0 (the
#  "License"); you may not use this file except in compliance
#  with the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing,
#  software distributed under the License is distributed on an
#  "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
#  KIND, either express or implied.  See the License for the
#  specific language governing permissions and limitations
#  under the License.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA

manifest_path = r"Z:/yufe/results/msfragger_ddaplus_paper/PXD041421/dda/fragpipe-files.fp-manifest"
dda_path = r"Z:/yufe/results/msfragger_ddaplus_paper/PXD041421/dda/combined_protein.tsv"
ddap_path = r"Z:/yufe/results/msfragger_ddaplus_paper/PXD041421/ddaplus/combined_protein.tsv"
dda_nombr_path = r"Z:/yufe/results/msfragger_ddaplus_paper/PXD041421/dda_nombr/combined_protein.tsv"
ddap_nombr_path = r"Z:/yufe/results/msfragger_ddaplus_paper/PXD041421/ddaplus_nombr/combined_protein.tsv"

manifest = pd.read_csv(manifest_path, sep='\t', index_col=False, header=None)
manifest.columns = ['File','condition','techrep','type']
manifest['File'] = manifest['File'].apply(lambda x: os.path.basename(x))
manifest['sample'] = manifest['condition'].str.cat(manifest['techrep'].map(str), sep='_')

def tidyup_combined_prot(data,):
    sample_cols = data.columns[data.columns.str.contains(' MaxLFQ Intensity')].tolist()
    meta_cols = ['Protein ID', 'Gene']
    data = data[meta_cols + sample_cols]
    data.columns = data.columns.str.replace(' MaxLFQ Intensity','')
    data = data.set_index(meta_cols)
    return data

dda = pd.read_csv(dda_path, sep="\t", index_col=False, na_values=['0', ''])
dda = tidyup_combined_prot(data = dda)

ddap = pd.read_csv(ddap_path, sep="\t", index_col=False, na_values=['0', ''])
ddap = tidyup_combined_prot(data = ddap)

dda_nombr = pd.read_csv(dda_nombr_path, sep="\t", index_col=False, na_values=['0', ''])
dda_nombr = tidyup_combined_prot(data = dda_nombr)

ddap_nombr = pd.read_csv(ddap_nombr_path, sep="\t", index_col=False, na_values=['0', ''])
ddap_nombr = tidyup_combined_prot(data = ddap_nombr)

ident = pd.DataFrame({'data':['DDA MBR-','DDA+ MBR-','DDA MBR+','DDA+ MBR+'],
                      'FDR':[0.01, 0.01, 0.01, 0.01],
                      'count':[dda_nombr.shape[0], ddap_nombr.shape[0], dda.shape[0], ddap.shape[0]]})

groups = manifest['condition'].unique()
cond_sample_dict = {}
for grp in groups:
    samples = manifest.loc[manifest['condition']==grp,'sample'].tolist()
    cond_sample_dict[grp] = samples

count_df = pd.DataFrame()
for data, datatype, mbrtype in zip(
    [dda, ddap, dda_nombr, ddap_nombr],
    ['DDA','DDA+','DDA','DDA+','DDA','DDA+'],
    ['MBR+','MBR+','MBR-','MBR-']):
    for grp in groups:
        sample_list = cond_sample_dict.get(grp)
        curr_data = data[sample_list]
        nval_count = curr_data.notna().sum(axis=1).value_counts()[[4,3,2,1]].tolist()
        nval_count_df = pd.DataFrame({
            'group':grp,
            'N':[4,3,2,1],
            'count':nval_count,
            'datatype':datatype,
            'MBR':mbrtype})
        count_df = pd.concat([count_df,nval_count_df], axis=0)

t = count_df.groupby(['group', 'datatype', 'MBR'])['count'].sum().reset_index()
t_pivot = t.pivot(index=['group', 'MBR'], columns='datatype', values='count').reset_index()
t_pivot['count_difference'] = t_pivot['DDA+'] - t_pivot['DDA']
t_pivot['count_difference_pct'] = t_pivot['count_difference'] * 100 / t_pivot['DDA']
print(t_pivot)

In [ ]:
# barplot

def barplot_N_ident_count(count_df, groups, prefix, fig_width, fig_height):
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    bar_width = 0.3
    n_bars = 6

    for datatype, color in zip(['DDA','DDA+'],['tab:blue','tab:orange']):
        for mbrtype in ['MBR-','MBR+']:
            if mbrtype=='MBR-':
                if datatype=='DDA+':
                    index = np.arange(len(groups)) * bar_width * n_bars + bar_width
                else:
                    index = np.arange(len(groups)) * bar_width * n_bars
            else:
                if datatype=='DDA+':
                    index = np.arange(len(count_df['group'].unique())) * bar_width * n_bars + bar_width*4
                else:
                    index = np.arange(len(count_df['group'].unique())) * bar_width * n_bars + bar_width*3
            
            for n, alpha in zip([4, 3, 2, 1], [1, 0.75, 0.5, 0.25]):
                curr_count_df = count_df.loc[(count_df['N']==n) & (count_df['datatype']==datatype) & (count_df['MBR']==mbrtype), ]
                if n==4:
                    acc_count = [0, 0, 0]
                else:
                    acc_count = count_df[(count_df['N']>=(n+1)) & (count_df['datatype']==datatype) & (count_df['MBR']==mbrtype)].groupby(['group'])['count'].agg('sum').tolist()
 
                plt.bar(index, curr_count_df.loc[:,'count'], bar_width, label = str(n), edgecolor = 'white', bottom = acc_count, alpha = alpha, color = color)
    
    plt.xlabel('')
    plt.ylabel('Number of quantified proteins')
    xlabel_list = []
    for g in groups:
        xlabel_list = xlabel_list + [g+'\nMBR-',g+'\nMBR+']
    plt.xticks(np.arange(len(groups)*2) * bar_width* 3 + bar_width - 0.5*bar_width, xlabel_list)
    
    handles, labels = ax.get_legend_handles_labels()

    dda_handles_labels = sorted([(label, handle) for label, handle in zip(labels[:4], handles[:4])], key=lambda x: int(x[0]), reverse=True)
    dda_plus_handles_labels = sorted([(label, handle) for label, handle in zip(labels[9:13], handles[9:13])], key=lambda x: int(x[0]), reverse=True)

    dda_labels, dda_handles = zip(*dda_handles_labels)
    dda_plus_labels, dda_plus_handles = zip(*dda_plus_handles_labels)
    
    fig.legend(dda_handles, dda_labels, loc="lower left", bbox_to_anchor=(0, -0.2),  ncol=4, fontsize="medium", title='DDA: Quantified in N out of 4')
    fig.legend(dda_plus_handles, dda_plus_labels, loc="lower right", bbox_to_anchor=(0.9, -0.2), ncol=4, fontsize="medium", title='DDA+: Quantified in N out of 4')
    
    ax.grid(False)
    plt.gcf().set_size_inches(fig_width, fig_height)
    plt.savefig(r"figure3_{}.pdf".format(prefix), bbox_inches='tight', pad_inches=0.1)


barplot_N_ident_count(count_df = count_df[count_df['group'].str.contains('A549')], groups = ['A549_1','A549_2','A549_3'], prefix='A549', fig_width = 8, fig_height = 4)
barplot_N_ident_count(count_df = count_df[count_df['group'].str.contains('K562')], groups = ['K562_1','K562_2','K562_3'], prefix='K562', fig_width = 8, fig_height = 4)

In [ ]:
# missing values

for useMBR in [False,True]:
    target_cell_columns = dda.columns[dda.columns.str.contains('A549_')]
    if useMBR:
        dda_tbl = dda[target_cell_columns].copy()
        dda_tbl = dda_tbl.loc[dda_tbl.isna().sum(axis=1) < len(target_cell_columns),]
        ddap_tbl = ddap[ddap.columns[ddap.columns.str.contains('A549_')]].copy()
        ddap_tbl = ddap_tbl.loc[ddap_tbl.isna().sum(axis=1) < len(target_cell_columns),]
        suffix = "MBR+"
    else:
        dda_tbl = dda_nombr[target_cell_columns].copy()
        dda_tbl = dda_tbl.loc[dda_tbl.isna().sum(axis=1) < len(target_cell_columns),]
        ddap_tbl = ddap_nombr[target_cell_columns].copy()
        ddap_tbl = ddap_tbl.loc[ddap_tbl.isna().sum(axis=1) < len(target_cell_columns),]
        suffix = "MBR-"

    dda_tbl[~dda_tbl.isna()] = 1
    dda_tbl[dda_tbl.isna()] = 0
    dda_mis_pct = dda_tbl.sum(axis=1, numeric_only=True)
    mis_pct_dda = dda_mis_pct.sort_values(ascending=True)
    dda_tbl = dda_tbl.loc[mis_pct_dda.index,:].copy()

    ddap_tbl[~ddap_tbl.isna()] = 1
    ddap_tbl[ddap_tbl.isna()] = 0
    ddap_mis_pct = ddap_tbl.sum(axis=1, numeric_only=True)
    mis_pct_ddap = ddap_mis_pct.sort_values(ascending=True)
    ddap_tbl = ddap_tbl.loc[mis_pct_ddap.index,:].copy()

    cmap_dict = {0: 'black', 1: 'orange'}
    cmap = ListedColormap([cmap_dict[i] for i in range(2)])

    fig, (ax1, ax2) = plt.subplots(1,2)
    sns.heatmap(data=dda_tbl, cmap=cmap, yticklabels=False, cbar=False, ax=ax1)
    ax1.set_title('DDA ' + suffix + ' (' + str(dda_tbl.shape[0]) + ' proteins)')
    ax1.set_ylabel('Protein index')

    sns.heatmap(data=ddap_tbl, cmap=cmap, yticklabels=False, cbar=False, ax=ax2, cbar_kws={'shrink': 0.2})
    ax2.set_title('DDA+ ' + suffix + ' (' + str(ddap_tbl.shape[0]) + ' proteins)')
    ax2.set_ylabel('')

    plt.gcf().set_size_inches(8, 4)
    plt.savefig(r"figure3_{}_A549.pdf".format(suffix), bbox_inches='tight', pad_inches=0.1)

    dda_zero_pct = dda_tbl.apply(lambda x: (x == 0).sum() * 100 / len(x), axis=0)
    ddap_zero_pct = ddap_tbl.apply(lambda x: (x == 0).sum() * 100 / len(x), axis=0)
    zero_pct_df = pd.DataFrame({
        'dda_zero_pct': dda_zero_pct,
        'ddap_zero_pct': ddap_zero_pct,
        'zero_pct_difference': ddap_zero_pct - dda_zero_pct
    })
    zero_pct_df.reset_index(inplace=True)
    zero_pct_df.rename(columns={'index': 'Sample'}, inplace=True)
    
    print("use MBR: " + str(useMBR))
    print(zero_pct_df)
    print(str(np.mean(zero_pct_df['dda_zero_pct'])) + "\t" + str(np.mean(zero_pct_df['ddap_zero_pct'])) + "\t" + str(np.mean(zero_pct_df['zero_pct_difference'])))


In [ ]:
# PCA

dda_prot_valid = set()
ddap_prot_valid = set()
biorep_list = manifest['condition'].unique()
for i in range(len(biorep_list)):
    biorep = biorep_list[i]
    curr_manifest = manifest[manifest['condition']==biorep]
    techrep = curr_manifest['sample'].tolist()
    if len(dda_prot_valid) == 0:
        dda_prot_valid = set(dda[dda[techrep].notna().sum(axis=1)>=2].reset_index()['Protein ID'])
    else:
        dda_prot_valid = set(dda[dda[techrep].notna().sum(axis=1)>=2].reset_index()['Protein ID']).intersection(dda_prot_valid)

    if len(ddap_prot_valid) == 0:
        ddap_prot_valid = set(ddap[ddap[techrep].notna().sum(axis=1)>=2].reset_index()['Protein ID'])
    else:
        ddap_prot_valid = set(ddap[ddap[techrep].notna().sum(axis=1)>=2].reset_index()['Protein ID']).intersection(ddap_prot_valid)

dda_prot = dda.reset_index()[dda.reset_index()['Protein ID'].isin(dda_prot_valid)]
ddap_prot = ddap.reset_index()[ddap.reset_index()['Protein ID'].isin(ddap_prot_valid)]

dda_prot = dda_prot.set_index(['Protein ID','Gene'])
ddap_prot = ddap_prot.set_index(['Protein ID','Gene'])

def plot_PCA(data, groups, datatype, bbox_to_anchor):
    markers = ["." , "," , "o" , "v" , "^" , "<", ">"]
    fig, ax = plt.subplots(figsize=(4, 4))
        
    X = data.T
    _,y = np.unique(np.array(groups), return_inverse=True)
    X = stats.zscore(X)
    pca = PCA()
    x_new = pca.fit_transform(X)
    explained = pca.explained_variance_ratio_
    
    for celltype, color in zip(['A549','K562'],['darkcyan','orange']):
        for i in [1,2,3]:
            techrep_name = '{}_{}'.format(celltype, str(i))
            marker = markers[i]
            techrep_index = [i for i in range(len(groups)) if techrep_name in groups[i] ]
        
            ax.scatter(x_new[techrep_index,0], x_new[techrep_index,1], c=color, label=techrep_name, s=150, marker=marker, edgecolors='black')
            ax.set_title(datatype)
            ax.set_xlabel('PC1: {}%'.format(round(explained[0]*100,2)))
            ax.set_ylabel('PC2: {}%'.format(round(explained[1]*100,2)))
            
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc="center right", bbox_to_anchor=bbox_to_anchor,  ncol=1, fontsize="small")

    ax.grid(False)
    plt.gcf().set_size_inches(4, 4)
    plt.savefig(r"figurS2_{}.pdf".format(datatype), bbox_inches='tight', pad_inches=0.1)

groups = [ manifest[manifest['sample'] == i]['condition'].tolist()[0] for i in data.columns ] 
dda_prot = dda_prot.dropna()
ddap_prot = ddap_prot.dropna()

plot_PCA(data = dda_prot, groups = groups, datatype = 'DDA', bbox_to_anchor=(1.2, 0.5))
plot_PCA(data = ddap_prot, groups = groups, datatype = 'DDA+', bbox_to_anchor=(1.2, 0.5))  


In [ ]:
# CV

biorep_list = manifest['condition'].unique()
dda_techrep_cv_df = pd.DataFrame()
ddap_techrep_cv_df = pd.DataFrame()
for i in range(len(biorep_list)):
    biorep = biorep_list[i]
    curr_manifest = manifest[manifest['condition']==biorep]
    techrep = curr_manifest['sample'].tolist()
    
    # DDA
    dda_prot = dda[techrep][dda[techrep].notna().sum(axis=1)>=2]
    curr_cv = dda_prot[techrep].apply(lambda x: np.std(x) / np.mean(x) * 100, axis=1).reset_index()
    curr_cv = curr_cv.rename(columns={0:'CV'})
    curr_cv['group'] = biorep
    dda_techrep_cv_df = pd.concat([dda_techrep_cv_df,curr_cv],axis=0)

    # DDA+
    ddap_prot = ddap[techrep][ddap[techrep].notna().sum(axis=1)>=2]
    curr_cv = ddap_prot[techrep].apply(lambda x: np.std(x) / np.mean(x) * 100, axis=1).reset_index()
    curr_cv = curr_cv.rename(columns={0:'CV'})
    curr_cv['group'] = biorep
    ddap_techrep_cv_df = pd.concat([ddap_techrep_cv_df,curr_cv],axis=0)

dda_techrep_cv_df['datatype'] = 'DDA'
ddap_techrep_cv_df['datatype'] = 'DDA+'
cv_df = pd.concat([ddap_techrep_cv_df, dda_techrep_cv_df], axis=0)

cv_df = cv_df.set_index(['Protein ID','Gene'])
cv_df['celltype'] = cv_df['group'].apply(lambda x: x.split('_')[0])

groups = cv_df['group'].unique()
all_cv_df = pd.DataFrame()
for i in range(len(groups)):
    grp = groups[i]
    cv_ddap = cv_df.loc[(cv_df['group']==grp) & (cv_df['datatype']=='DDA+'),].copy()
    cv_dda = cv_df.loc[(cv_df['group']==grp) & (cv_df['datatype']=='DDA'),].copy()
    common_ids = cv_ddap.index.intersection(cv_dda.index)
    ddap_unique = cv_ddap.index.difference(cv_dda.index)
    dda_unique = cv_dda.index.difference(cv_ddap.index)

    cv_ddap['ctg'] = 'DDA+ common'
    cv_ddap.loc[cv_ddap.index.isin(ddap_unique), 'ctg'] = 'DDA+ unique'

    cv_dda['ctg'] = 'DDA common'
    cv_dda.loc[cv_dda.index.isin(dda_unique), 'ctg'] = 'DDA unique'

    combined_cv = pd.concat([cv_ddap, cv_dda], axis=0)
    all_cv_df = pd.concat([all_cv_df, combined_cv], axis=0)

all_cv_df = all_cv_df.reset_index()

def show_values_on_bars(axs, font_size):
    def _show_on_single_plot(ax):
        val_list = [ x for x in ax.containers[0].datavalues ]
        if len(ax.containers)>0:
            for i in range(1,len(ax.containers)):
                tmp_list = [ x for x in ax.containers[i].datavalues ]
                val_list.extend(tmp_list)
        for i in range(len(ax.patches)):
            if i >= len(val_list):
                break

            p = ax.patches[i]
            _x = p.get_x()
            _y = p.get_y() + p.get_height()

            value = val_list[i]
            ax.text(_x, _y, int(value), ha="left", color='black', fontsize=font_size)

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)


def group_barplot(cv_df, prefix, font_size, bbox_to_anchor, fig_width, fig_height):
    custom_params = {"axes.spines.right": True, "axes.spines.top": True, "axes.linewidth": 0.75}
    sns.set_theme(style="ticks", rc=custom_params, font="Arial")
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(fig_width, fig_height), gridspec_kw={'width_ratios': [6, 1]}, constrained_layout = True)
    ax1 = axes[0]
    ax2 = axes[1]

    bar_width = 0.75
    PROPS = {
        'boxprops':{'edgecolor':'black'},
        'medianprops':{'color':'black'},
        'whiskerprops':{'color':'black'},
        'capprops':{'color':'black'},
        'flierprops':{'markersize':2,'marker':"x",'markerfacecolor':'black','color':'black'}
    }

    # barplot
    sns.boxplot(x = cv_df['CV'],
                y = cv_df['group'],
                hue = cv_df['ctg'],
                hue_order = ['DDA unique', 'DDA common', 'DDA+ common', 'DDA+ unique'],
                palette = {"DDA unique": "#4285F4","DDA common": "#34A853","DDA+ common": "#EA4335","DDA+ unique": "orange"},
                width = bar_width,
                showfliers = False,
                whis = 1.5,
                linewidth = 0.75,
                saturation = 1,
                ax = ax1,
                **PROPS)
    ax1.legend_ = None
    ax1.set_xlabel('CV (%)')
    ax1.set_ylabel('')
    ax1.tick_params(width=0.75, color='black')

    # show number
    cv_group_count = cv_df[['group','ctg','Protein ID']].groupby(['group','ctg']).agg(lambda x: len(x)).reset_index()
    sns.barplot(x = cv_group_count["Protein ID"],
                y = cv_group_count["group"],
                hue = cv_group_count["ctg"],
                hue_order = ['DDA unique', 'DDA common', 'DDA+ common', 'DDA+ unique'],
                palette = {"DDA unique": "white","DDA common": "white","DDA+ common": "white","DDA+ unique": "white"},
                width = bar_width,
                errorbar = None,
                saturation = 1,
                ax = ax2)
    ax2.legend_ = None
    ax2.yaxis.set_visible(False)
    ax2.xaxis.set_visible(False)
    ax2.spines[['right','left','bottom','top']].set_visible(False)
    ax2.set_xlabel('Count')
    ax2.set_ylabel('')
    ax2.tick_params(width=0.75, color='black')
    show_values_on_bars(ax2, font_size=font_size)

    handles, labels = ax1.get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=bbox_to_anchor,  ncol=2, fontsize="small")

    plt.gcf().set_size_inches(fig_width, fig_height)
    plt.savefig(r"{}_unique_common_boxplot.pdf".format(prefix), bbox_inches='tight', pad_inches=0.1)


group_barplot(cv_df = all_cv_df[all_cv_df['celltype']=='A549'], prefix='A549', font_size=10, bbox_to_anchor=(0.5, -0.17), fig_width=6, fig_height=3)
group_barplot(cv_df = all_cv_df[all_cv_df['celltype']=='K562'], prefix='K562', font_size=10, bbox_to_anchor=(0.5, -0.17), fig_width=6, fig_height=3)
